In [4]:
import sys
import os
import time
import multiprocessing
import numpy as np
from numba import njit, prange

# ==============================================================================
# 1. GESTIÓN DE ARGUMENTOS
# ==============================================================================
# Valor por defecto (50 millones)
value = 5 * 10**7 

# Si recibimos argumentos desde SLURM, actualizamos value
if len(sys.argv) > 1:
    try:
        param = float(sys.argv[1])
        value = int(param)
    except ValueError:
        pass

# ==============================================================================
# 2. DEFINICIÓN DE FUNCIONES
# ==============================================================================

# --- A) CÓDIGO ORIGINAL (SIN OPTIMIZAR) ---
def reduc_operation(A):
    """Compute the sum of the elements of Array A."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# --- B) FUNCIONES PARA MULTIPROCESSING ---
def worker_sum(chunk):
    return np.sum(chunk)

def run_multiprocessing_fixed(array, n_procs):
    # Dividimos el array explícitamente
    chunks = np.array_split(array, n_procs)
    with multiprocessing.Pool(processes=n_procs) as pool:
        results = pool.map(worker_sum, chunks)
    return sum(results)

# --- C) FUNCIONES PARA NUMBA ---
@njit
def reduc_numba_seq(A):
    s = 0.0
    for i in range(A.size):
        s += A[i]
    return s

@njit(parallel=True)
def reduc_numba_par(A):
    s = 0.0
    for i in prange(A.size):
        s += A[i]
    return s

# ==============================================================================
# 3. EJECUCIÓN PRINCIPAL
# ==============================================================================
if __name__ == '__main__':
    # Generación de datos
    print(f"Generando array de {value} elementos...")
    X = np.random.rand(value)
    
    # Lectura del entorno SLURM
    slurm_cores = int(os.environ.get('OMP_NUM_THREADS', 4))
    
    print(f"\n====== INFORME DE EJECUCIÓN (SLURM CORES: {slurm_cores}) ======\n")

    # --------------------------------------------------------------------------
    # APARTADO 0: CÓDIGO ORIGINAL Y NUMPY (INTEGRADO)
    # --------------------------------------------------------------------------
    # PROTECCIÓN: Solo ejecutamos el código lento si el array es manejable.
    # Si es 10^8 o 10^9, saltamos esta parte para no bloquear el clúster.
    if value <= 5 * 10**7:
        print("--- BASE: Comparativa Python Puro vs Optimizaciones ---")
        
        # 1. Python Puro (reduc_operation)
        # Usamos time.time() en lugar de %timeit para que sea limpio en el log de SLURM
        start = time.time()
        res_pure = reduc_operation(X)
        end = time.time()
        print(f"   [Python Loop] Tiempo: {end - start:.6f} s | Resultado: {res_pure:.2f}")

    # --------------------------------------------------------------------------
    # APARTADO A: MULTIPROCESSING
    # (Requisito: Mostrar siempre 2 y 4 procesos)
    # --------------------------------------------------------------------------
    print("--- APARTADO A: Multiprocessing (Comparativa 2 vs 4 procesos) ---")
    
    # Prueba con 2 procesos
    start = time.time()
    res_2 = run_multiprocessing_fixed(X, 2)
    end = time.time()
    print(f"   [2 Procesos] Tiempo: {end - start:.6f} s")

    # Prueba con 4 procesos
    start = time.time()
    res_4 = run_multiprocessing_fixed(X, 4)
    end = time.time()
    print(f"   [4 Procesos] Tiempo: {end - start:.6f} s")
    
    print("") 

    # --------------------------------------------------------------------------
    # APARTADO B: NUMBA
    # (Requisito: Secuencial vs Paralelo Dinámico)
    # --------------------------------------------------------------------------
    print(f"--- APARTADO B: Numba (Secuencial vs Paralelo con {slurm_cores} hilos) ---")
    
    # 1. Secuencial
    reduc_numba_seq(X[:100]) # Warm-up
    start = time.time()
    res_seq = reduc_numba_seq(X)
    end = time.time()
    print(f"   [Secuencial] Tiempo: {end - start:.6f} s")

    # 2. Paralelo
    reduc_numba_par(X[:100]) # Warm-up
    start = time.time()
    res_par = reduc_numba_par(X)
    end = time.time()
    print(f"   [Paralelo]   Tiempo: {end - start:.6f} s")
    
    print("\n" + "="*70 + "\n")

Generando array de 50000000 elementos...

====== INFORME DE EJECUCIÓN (SLURM CORES: 4) ======

--- BASE: Comparativa Python Puro vs Optimizaciones ---
   [Python Loop] Tiempo: 5.262784 s | Resultado: 25002830.20
--- APARTADO A: Multiprocessing (Comparativa 2 vs 4 procesos) ---
   [2 Procesos] Tiempo: 0.308509 s
   [4 Procesos] Tiempo: 0.314038 s

--- APARTADO B: Numba (Secuencial vs Paralelo con 4 hilos) ---
   [Secuencial] Tiempo: 0.050871 s
   [Paralelo]   Tiempo: 0.011658 s




==============================================================
    INICIANDO BATERÍA PARA TAMAÑO: 50000000
==============================================================
 
>>> Configuración SLURM: 1 Hilos <<<
Generando array de 50000000 elementos...

====== INFORME DE EJECUCIÓN (SLURM CORES: 1) ======

--- BASE: Comparativa Python Puro vs Optimizaciones ---
   [Python Loop] Tiempo: 8.630882 s | Resultado: 25001657.95
--- APARTADO A: Multiprocessing (Comparativa 2 vs 4 procesos) ---
   [2 Procesos] Tiempo: 0.813041 s
   [4 Procesos] Tiempo: 0.681264 s

--- APARTADO B: Numba (Secuencial vs Paralelo con 1 hilos) ---
   [Secuencial] Tiempo: 0.057691 s
   [Paralelo]   Tiempo: 0.023360 s

======================================================================

 
>>> Configuración SLURM: 2 Hilos <<<
Generando array de 50000000 elementos...

====== INFORME DE EJECUCIÓN (SLURM CORES: 2) ======

--- BASE: Comparativa Python Puro vs Optimizaciones ---
   [Python Loop] Tiempo: 8.547583 s | Resultado: 24998982.20
--- APARTADO A: Multiprocessing (Comparativa 2 vs 4 procesos) ---
   [2 Procesos] Tiempo: 0.772918 s
   [4 Procesos] Tiempo: 0.682315 s

--- APARTADO B: Numba (Secuencial vs Paralelo con 2 hilos) ---
   [Secuencial] Tiempo: 0.059373 s
   [Paralelo]   Tiempo: 0.033893 s

======================================================================

 
>>> Configuración SLURM: 4 Hilos <<<
Generando array de 50000000 elementos...

====== INFORME DE EJECUCIÓN (SLURM CORES: 4) ======

--- BASE: Comparativa Python Puro vs Optimizaciones ---
   [Python Loop] Tiempo: 8.843400 s | Resultado: 25002128.40
--- APARTADO A: Multiprocessing (Comparativa 2 vs 4 procesos) ---
   [2 Procesos] Tiempo: 0.774250 s
   [4 Procesos] Tiempo: 0.680848 s

--- APARTADO B: Numba (Secuencial vs Paralelo con 4 hilos) ---
   [Secuencial] Tiempo: 0.059180 s
   [Paralelo]   Tiempo: 0.015924 s

======================================================================

 
>>> Configuración SLURM: 8 Hilos <<<
Generando array de 50000000 elementos...

====== INFORME DE EJECUCIÓN (SLURM CORES: 8) ======

--- BASE: Comparativa Python Puro vs Optimizaciones ---
   [Python Loop] Tiempo: 8.482277 s | Resultado: 25000979.04
--- APARTADO A: Multiprocessing (Comparativa 2 vs 4 procesos) ---
   [2 Procesos] Tiempo: 0.766490 s
   [4 Procesos] Tiempo: 0.681821 s

--- APARTADO B: Numba (Secuencial vs Paralelo con 8 hilos) ---
   [Secuencial] Tiempo: 0.059399 s
   [Paralelo]   Tiempo: 0.015151 s

======================================================================

 
==============================================================
    INICIANDO BATERÍA PARA TAMAÑO: 100000000
==============================================================
 
>>> Configuración SLURM: 1 Hilos <<<
Generando array de 100000000 elementos...

====== INFORME DE EJECUCIÓN (SLURM CORES: 1) ======

--- APARTADO A: Multiprocessing (Comparativa 2 vs 4 procesos) ---
   [2 Procesos] Tiempo: 1.512838 s
   [4 Procesos] Tiempo: 1.367180 s

--- APARTADO B: Numba (Secuencial vs Paralelo con 1 hilos) ---
   [Secuencial] Tiempo: 0.117776 s
   [Paralelo]   Tiempo: 0.026657 s

======================================================================

 
>>> Configuración SLURM: 2 Hilos <<<
Generando array de 100000000 elementos...

====== INFORME DE EJECUCIÓN (SLURM CORES: 2) ======

--- APARTADO A: Multiprocessing (Comparativa 2 vs 4 procesos) ---
   [2 Procesos] Tiempo: 1.471374 s
   [4 Procesos] Tiempo: 1.367693 s

--- APARTADO B: Numba (Secuencial vs Paralelo con 2 hilos) ---
   [Secuencial] Tiempo: 0.117588 s
   [Paralelo]   Tiempo: 0.032374 s

======================================================================

 
>>> Configuración SLURM: 4 Hilos <<<
Generando array de 100000000 elementos...

====== INFORME DE EJECUCIÓN (SLURM CORES: 4) ======

--- APARTADO A: Multiprocessing (Comparativa 2 vs 4 procesos) ---
   [2 Procesos] Tiempo: 1.512356 s
   [4 Procesos] Tiempo: 1.367215 s

--- APARTADO B: Numba (Secuencial vs Paralelo con 4 hilos) ---
   [Secuencial] Tiempo: 0.117752 s
   [Paralelo]   Tiempo: 0.046632 s

======================================================================

 
>>> Configuración SLURM: 8 Hilos <<<
Generando array de 100000000 elementos...

====== INFORME DE EJECUCIÓN (SLURM CORES: 8) ======

--- APARTADO A: Multiprocessing (Comparativa 2 vs 4 procesos) ---
   [2 Procesos] Tiempo: 1.517367 s
   [4 Procesos] Tiempo: 1.367669 s

--- APARTADO B: Numba (Secuencial vs Paralelo con 8 hilos) ---
   [Secuencial] Tiempo: 0.117567 s
   [Paralelo]   Tiempo: 0.024220 s

======================================================================

 
==============================================================
    INICIANDO BATERÍA PARA TAMAÑO: 1000000000
==============================================================
 
>>> Configuración SLURM: 1 Hilos <<<
Generando array de 1000000000 elementos...

====== INFORME DE EJECUCIÓN (SLURM CORES: 1) ======

--- APARTADO A: Multiprocessing (Comparativa 2 vs 4 procesos) ---
   [2 Procesos] Tiempo: 15.865407 s
   [4 Procesos] Tiempo: 13.394336 s

--- APARTADO B: Numba (Secuencial vs Paralelo con 1 hilos) ---
   [Secuencial] Tiempo: 1.166268 s
   [Paralelo]   Tiempo: 0.321492 s

======================================================================

 
>>> Configuración SLURM: 2 Hilos <<<
Generando array de 1000000000 elementos...

====== INFORME DE EJECUCIÓN (SLURM CORES: 2) ======

--- APARTADO A: Multiprocessing (Comparativa 2 vs 4 procesos) ---
   [2 Procesos] Tiempo: 15.640908 s
   [4 Procesos] Tiempo: 14.026332 s

--- APARTADO B: Numba (Secuencial vs Paralelo con 2 hilos) ---
   [Secuencial] Tiempo: 1.166327 s
   [Paralelo]   Tiempo: 0.230192 s

======================================================================

 
>>> Configuración SLURM: 4 Hilos <<<
Generando array de 1000000000 elementos...

====== INFORME DE EJECUCIÓN (SLURM CORES: 4) ======

--- APARTADO A: Multiprocessing (Comparativa 2 vs 4 procesos) ---
   [2 Procesos] Tiempo: 15.908798 s
   [4 Procesos] Tiempo: 13.631452 s

--- APARTADO B: Numba (Secuencial vs Paralelo con 4 hilos) ---
   [Secuencial] Tiempo: 1.166504 s
   [Paralelo]   Tiempo: 0.264453 s

======================================================================

 
>>> Configuración SLURM: 8 Hilos <<<
Generando array de 1000000000 elementos...

====== INFORME DE EJECUCIÓN (SLURM CORES: 8) ======

--- APARTADO A: Multiprocessing (Comparativa 2 vs 4 procesos) ---
   [2 Procesos] Tiempo: 15.916457 s
   [4 Procesos] Tiempo: 13.427435 s

--- APARTADO B: Numba (Secuencial vs Paralelo con 8 hilos) ---
   [Secuencial] Tiempo: 1.165216 s
   [Paralelo]   Tiempo: 0.313930 s

======================================================================

Trabajo finalizado.

1. Código Original (Python Loop Puro): (50M elementos): Tarda unos 8.5 - 8.8 segundos. Lo he hecho con 50M de elementos por si acaso se saturaba el clúster con 10^8 y 10^9 elementos. Comparado con Numba (0.05s), es unas 150 veces más lento.

2. Multiprocessing con Pool. Es más rápido que el código original, pero mucho más lento que Numba. Escalabilidad pobre: Al pasar de 2 a 4 procesos, el tiempo baja un poco (de 15.8s a 13.4s), pero no se reduce a la mitad. La operación de suma es tan rápida para la CPU que se tarda más tiempo moviendo los datos a los procesos que sumándolos. El sistema está limitado por la velocidad de la memoria RAM, no por la potencia de cálculo.

3. Numba. Este es el claro vencedor. Analicemos sus dos vertientes: Es unas 10-12 veces más rápido que Multiprocessing usando un solo núcleo. Numba compila la función Python a código máquina optimizado (como C o Fortran) antes de ejecutarla (JIT). Elimina todas las comprobaciones del intérprete de Python.

Numba paralelo es la opción más rápida, ~50 veces más rápido que Multiprocessing. Añadir más núcleos no mejora el tiempo, a veces lo empeora. Esto confirma que hemos alcanzado el límite de velocidad de la memoria RAM (Memory Bandwidth Bound). 2 o 4 núcleos son suficientes para sumar tan rápido como la memoria entrega los datos; poner 8 núcleos solo crea un "atasco" en el acceso a los datos.